<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week7.do.txt --no_mako -->
<!-- dom:TITLE: March 6-10, 2023: Quantum Computing, Quantum Machine Learning and Quantum Information Theories -->

# March 6-10, 2023: Quantum Computing, Quantum Machine Learning and Quantum Information Theories
**Morten Hjorth-Jensen**, Department of Physics, University of Oslo and Department of Physics and Astronomy and Facility for Rare Isotope Beams, Michigan State University

Date: **March 6-10**

## Algorithms for solving quantum mechanical problems, quantum gates and circuits

1. Introduction to [Qiskit](https://qiskit.org/)

2. Defining one-, two- and three-qubit gates

3. Setting up quantum circuits and simple algorithms

4. VQE, Variational Quantum Eigensolver

5. Selected Hamiltonians